In [1]:
import json
import requests
import asyncio
import aiohttp
import itertools

In [2]:
default_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0'
}

async def query_page(session, page, headers=default_headers):
    async with session.get(page, headers=headers) as response:
        return await response.text()
    
async def query_all_pages(pages, headers=default_headers):
    queries = []
    async with aiohttp.ClientSession() as session:
        queries = await asyncio.gather(*[query_page(session, page, headers) for page in pages], return_exceptions=True)
    return [q for q in queries]

In [80]:
req = requests.get('https://touch.dofusbook.net/items/touch/search/equipment?page=1')
first_page = json.loads(req.text)
pages = [first_page]
page_count = first_page["pages"]

query_res = await query_all_pages([f'https://touch.dofusbook.net/items/touch/search/equipment?page={i}' for i in range(1, page_count+1)])
pages.extend([json.loads(res) for res in query_res])

items = list(itertools.chain.from_iterable([p["data"] for p in pages]))

In [81]:
with open('item_db.json', 'w') as f:
    json.dump(items, f)

In [82]:
import pandas as pd
items = pd.DataFrame(items)
items

,id,official,picture,level,cloth_id,choose_effect,give_boost,cannot_fm,category_id,subcategory_id,...,png_color1,png_color2,png_color3,size,cameleon,slug,effects,constraints,ingredients,weapon
0,5190,16382,171114,200,NaN,0,0,0,12,None,...,None,None,None,NaN,NaN,5190-amulette-de-shokkoth,"[{'item_id': 5190, 'id': 130, 'name': 'vi', 't...",[],"[{'item_id': 5190, 'id': 11883, 'name': 'Condy...",None
1,5189,16534,171194,200,NaN,0,0,0,16,None,...,None,None,None,NaN,NaN,5189-scaramouchapeau,"[{'item_id': 5189, 'id': 330, 'name': 'ii', 't...","[[{'item_id': 5189, 'id': 50, 'name': 'pa', 's...","[{'item_id': 5189, 'id': 11522, 'name': 'Fragm...",None
2,5188,16186,171024,200,NaN,0,0,0,19,None,...,None,None,None,NaN,NaN,5188-chaussons-de-macrab,"[{'item_id': 5188, 'id': 130, 'name': 'vi', 't...","[[{'item_id': 5188, 'id': 50, 'name': 'pa', 's...","[{'item_id': 5188, 'id': 8809, 'name': 'Culott...",None
3,5187,16190,171025,200,NaN,0,0,0,16,None,...,None,None,None,NaN,NaN,5187-masque-de-funespadon,"[{'item_id': 5187, 'id': 130, 'name': 'vi', 't...","[[{'item_id': 5187, 'id': 50, 'name': 'pa', 's...","[{'item_id': 5187, 'id': 11527, 'name': 'Peau ...",None
4,5186,16354,171101,200,NaN,0,0,0,15,None,...,None,None,None,NaN,NaN,5186-protopagne,"[{'item_id': 5186, 'id': 130, 'name': 'vi', 't...","[[{'item_id': 5186, 'id': 50, 'name': 'pa', 's...","[{'item_id': 5186, 'id': 8808, 'name': 'Étoffe...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2386,1406,11812,9206,1,NaN,0,0,0,13,None,...,None,None,None,NaN,NaN,1406-elianneau,"[{'item_id': 1406, 'id': 700, 'name': 'ti', 't...",[],[],None
2387,1199,10906,82056,1,NaN,0,0,0,14,None,...,None,None,None,NaN,0.0,1199-bouclier-draconique,"[{'item_id': 1199, 'id': 620, 'name': 'pap', '...",[],[],None
2388,117,789,10009,1,NaN,0,0,0,15,None,...,None,None,None,NaN,NaN,117-ceinture-du-kobeer,"[{'item_id': 117, 'id': 140, 'name': 'fo', 'ty...",[],[],None
2389,1142,10414,16251,1,NaN,0,0,0,16,None,...,None,None,None,NaN,0.0,1142-couronne-koninginnedag,"[{'item_id': 1142, 'id': 200, 'name': 'cc', 't...",[],[],None


In [6]:
touch_domain = "https://www.dofus-touch.com"

In [7]:
from bs4 import BeautifulSoup
import re

professions = {}
profession_pages = await query_all_pages(f'https://www.dofus-touch.com/fr/mmorpg/encyclopedie/metiers?display=table&page={i+1}' for i in [0, 1])
for profession_page in profession_pages[0:1]:
    soup = BeautifulSoup(profession_page, 'html.parser')

    # The page is composed of a list of professions. First get the panel where they are located
    main_panel = soup.find("div", "ak-container ak-main-center")

    # The look for each item
    profession_items = main_panel.find_all("td", "item-name")
    assert(len(profession_items) > 0)

    for profession_item in profession_items:
        # The profession item is composed of the "item-name" containing the name in the string and a child "a" link
        professions[profession_item.string.strip()] = {"link":profession_item.find("a", href=re.compile("encyclopedie/metiers/")).get("href")}


In [8]:
professions

{'Alchimiste': {'link': '/fr/mmorpg/encyclopedie/metiers/26-alchimiste'},
 'Bijoutier': {'link': '/fr/mmorpg/encyclopedie/metiers/16-bijoutier'},
 'Boucher': {'link': '/fr/mmorpg/encyclopedie/metiers/56-boucher'},
 'Boulanger': {'link': '/fr/mmorpg/encyclopedie/metiers/25-boulanger'},
 'Bricoleur': {'link': '/fr/mmorpg/encyclopedie/metiers/65-bricoleur'},
 'Bûcheron': {'link': '/fr/mmorpg/encyclopedie/metiers/2-bucheron'},
 'Chasseur': {'link': '/fr/mmorpg/encyclopedie/metiers/41-chasseur'},
 'Cordomage': {'link': '/fr/mmorpg/encyclopedie/metiers/62-cordomage'},
 'Cordonnier': {'link': '/fr/mmorpg/encyclopedie/metiers/15-cordonnier'},
 'Costumage': {'link': '/fr/mmorpg/encyclopedie/metiers/64-costumage'},
 'Façonneur': {'link': '/fr/mmorpg/encyclopedie/metiers/60-faconneur'},
 "Forgemage d'Epées": {'link': '/fr/mmorpg/encyclopedie/metiers/44-forgemage-epees'},
 'Forgemage de Dagues': {'link': '/fr/mmorpg/encyclopedie/metiers/43-forgemage-dagues'},
 'Forgemage de Haches': {'link': '/fr/

In [9]:
profession_pages

['<!DOCTYPE HTML>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="fr" xml:lang="fr">\n<head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n  <title>Métiers - Encyclopédie DOFUS -  DOFUS Touch, le MMORPG ultime sur mobile</title>\n  <meta name="description" content="Maîtrisez jusqu\'à trois métiers simultanément parmi les vingt-deux existants ! En combinant récolte et artisanat, vous créerez vos propres objets et équipements." />\n  <meta name="keywords" content="ARTISANAT, METIERS,  COLLECTE, FORGEMAGIE, ALCHIMISTE, BUCHERON, MINEUR, PAYSAN, PECHEUR, BIJOUTIER, BOUCHER, BOULANGER, BRICOLEUR, CHASSEUR, CORDONNIER, FORGEUR, POISSONNIER, SCULPTEUR, TAILLEUR, DOFUS, ENCYCLOPEDIE, MMORPG, JEU EN LIGNE" />\n  <meta name="Identifier-URL" content="www.dofus-touch.com" />\n  <meta name="language" content="fr" />\n  <meta http-equiv="imagetoolbar" content="no" />\n  <meta property="fb:admins" content="100000245210540"/>\n  <meta property="og:type" content="website"/

In [10]:
profession_pages = await query_all_pages([l for l in [touch_domain + l["link"] + "/recipes" for l in professions.values()]])

all_recipes = []
for [[profession_name, profession], profession_first_page] in zip(professions.items(), profession_pages): 
    page_id = 1
    page = profession_first_page
    profession["recipes"] = []
    while (page != None):
        profession_page = BeautifulSoup(page, 'html.parser')

        # There should be a tab content panel containing tabs for harvests and recipes
        tab_panel = profession_page.find("div", "tab-content") 

        # Get the currently active tab, which should be the recipes. Inactive tabs will have the "ak-tab hide" class
        recipe_tabs = [f for f in tab_panel.find_all("div", "ak-tab") if "hide" not in f["class"]]
        
        # We only expect magus items to be without a tab
        if (len(recipe_tabs) == 0):
            assert("mage" in profession_name)
            page = None
            continue

        # Go the the tab body
        tab_body = recipe_tabs[0].find("tbody")

        # This body should have a list of "tr" elements with the recipe dynamically loadable
        recipes_tr = tab_body.find_all("tr", attrs={"ajax-details-url":re.compile("objets/recette/")})
        
        for recipe_tr in recipes_tr:
            recipe = {}

            # The recipe contains a link to the crafted resource
            link_html  = recipe_tr.find("a", href=re.compile("encyclopedie/"))

            # Row contains the name of the object, then the level. link_html is the child of the name container, which is sibling of the level's container
            level_html = link_html.parent.findNextSibling("td")

            # The detail url is of form "/en/mmorpg/encyclopedia/items/recipe/286", 286 being the item id
            recipe["detail_url"]    = recipe_tr.get("ajax-details-url")
            recipe["object_id"]     = int(recipe["detail_url"].split("/")[-1])
            recipe["object_name"]   = link_html.string
            recipe["link"]          = link_html.get("href")
            recipe["level"]         = int(level_html.string)
            recipe["profession"]    = profession_name

            all_recipes.append(recipe)
            profession["recipes"].append(recipe)

        page_id += 1
        page = None
        next_page_link = profession_page.find("link", rel="next", href=re.compile(profession["link"]))
        if next_page_link != None:
            page = requests.get(touch_domain + next_page_link.get("href"), headers=default_headers).text

In [11]:
profession_page

<!DOCTYPE HTML>

<html lang="fr" xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>Paysan - Métiers - Encyclopédie DOFUS -  DOFUS Touch, le MMORPG ultime sur mobile</title>
<meta content="Le paysan consacre une partie de son temps à faucher des céréales dans les champs de la province d'Amakna, et l'autre partie à moudre le fruit de ses récoltes pour vendre ses farines aux boulangers les plus offrants." name="description"/>
<meta content="PAYSAN, METIERS, ARTISANAT, RECOLTES, RESSOURCES, RECETTES, DOFUS, ENCYCLOPEDIE, MMORPG, JEU EN LIGNE" name="keywords"/>
<meta content="www.dofus-touch.com" name="Identifier-URL"/>
<meta content="fr" name="language"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="100000245210540" property="fb:admins"/>
<meta content="website" property="og:type"/>
<meta content="fr_FR" property="og:locale"/>
<meta content="Paysan - Métiers - Encyclopédie DOFUS -  DOFUS T

In [12]:
all_recipes

[{'detail_url': '/fr/mmorpg/encyclopedie/objets/recette/1642',
  'object_id': 1642,
  'object_name': "Potion d'oubli de métier : Forgemage de Dagues",
  'link': '/fr/mmorpg/encyclopedie/consommables/1642-potion-oubli-metier-forgemage-dagues',
  'level': 1,
  'profession': 'Alchimiste'},
 {'detail_url': '/fr/mmorpg/encyclopedie/objets/recette/1643',
  'object_id': 1643,
  'object_name': "Potion d'oubli de métier : Forgemage de Haches",
  'link': '/fr/mmorpg/encyclopedie/consommables/1643-potion-oubli-metier-forgemage-haches',
  'level': 1,
  'profession': 'Alchimiste'},
 {'detail_url': '/fr/mmorpg/encyclopedie/objets/recette/1644',
  'object_id': 1644,
  'object_name': "Potion d'oubli de métier : Forgemage de Marteaux",
  'link': '/fr/mmorpg/encyclopedie/consommables/1644-potion-oubli-metier-forgemage-marteaux',
  'level': 1,
  'profession': 'Alchimiste'},
 {'detail_url': '/fr/mmorpg/encyclopedie/objets/recette/1645',
  'object_id': 1645,
  'object_name': "Potion d'oubli de métier : For

In [13]:
len(all_recipes)

1839

In [62]:
professions

{'Alchimiste': {'link': '/fr/mmorpg/encyclopedie/metiers/26-alchimiste',
  'recipes': [{'detail_url': '/fr/mmorpg/encyclopedie/objets/recette/1642',
    'object_id': 1642,
    'object_name': "Potion d'oubli de métier : Forgemage de Dagues",
    'link': '/fr/mmorpg/encyclopedie/consommables/1642-potion-oubli-metier-forgemage-dagues',
    'level': 1,
    'profession': 'Alchimiste',
    'ingredients': [{'count': '1',
      'link': '/fr/mmorpg/encyclopedie/armes/340-leurnettes',
      'name': 'Leurnettes'},
     {'count': '2',
      'link': '/fr/mmorpg/encyclopedie/consommables/793-potion-vieillesse',
      'name': 'Potion de Vieillesse'}]},
   {'detail_url': '/fr/mmorpg/encyclopedie/objets/recette/1643',
    'object_id': 1643,
    'object_name': "Potion d'oubli de métier : Forgemage de Haches",
    'link': '/fr/mmorpg/encyclopedie/consommables/1643-potion-oubli-metier-forgemage-haches',
    'level': 1,
    'profession': 'Alchimiste',
    'ingredients': [{'count': '2',
      'link': '/fr/m

In [16]:
request_count = 0
list_404 = []
for [name, profession] in professions.items():
    
    print(name)
    recipes = profession["recipes"]
    recipe_fetch_headers = {
        "Accept"        :"*/*",
        "X-Requested-With": "XMLHttpRequest",
        "Referer"       :profession["link"] + "/recipes",
        "Sec-Fetch-Dest":"empty",
        "Sec-Fetch-Mode":"cors",
        "Sec-Fetch-Site":"same-origin",
        "User-Agent"    :"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0"
    }

    request_count += len(recipes) 
    all_recipe_json = await query_all_pages([f"https://www.dofus-touch.com/fr/mmorpg/encyclopedie/objets/recette/" + str(r["object_id"]) for r in recipes], headers=recipe_fetch_headers)
    all_recipe_html = [json.loads(j) for j in all_recipe_json]

    recipe["ingredients"] = []

    for [recipe, html] in zip(recipes, all_recipe_html):
        # For some reason some recepy html are empty. Should probably tell ankama...
        if html == "":
            continue

        soup = BeautifulSoup(html, "html.parser")

        # We also have 404 errors on some object links. Ankama fix link plz
        if (soup.find("div", "ak-404")):
            list_404.append([recipe, soup])
            continue

        # This documennt is a grid of objects. First find the container of the grid
        grid_container = soup.find("div", "ak-content-list")

        # Now we can get all the items in the grid. They are all of the "ak-list-element" class
        ingredients_html = grid_container.find_all("div", "ak-list-element")

        ingredients = []

        for ingredient_html in ingredients_html:
            # The item count is embedded in a text in front of the content : e.g "10 x" contained in a "ak-front" element
            item_count_text = ingredient_html.find("div", "ak-front")
            item_count = re.match("(\d+) x", item_count_text.string.strip()).group(1)
            
            # Now go and get the item properties, they are all in a div under the item
            content = ingredient_html.find("div", "ak-content")

            ingredient = {}

            # Gather the data.
            ingredient["count"] = item_count
            ingredient["link"] = content.find("a").get("href")
            ingredient["name"] = content.find("span", "ak-linker").string
            ingredients.append(ingredient)

        recipe["ingredients"] = ingredients
        
    # Force a wait to avoid being blocked out of the website. Limit rate of cloudfront is 5 minutes, so we assume we need to wait that much
    # 500 is arbitrary, we can probably spam our way to a precise number but that should do for now
    if (request_count > 500):
        request_count = 0
        
        await asyncio.sleep(5 * 60 + 1)

Alchimiste
Bijoutier
Boucher
Boulanger
Bricoleur
Bûcheron
Chasseur
Cordomage
Cordonnier
Costumage
Façonneur
Forgemage d'Epées
Forgemage de Dagues
Forgemage de Haches
Forgemage de Marteaux
Forgemage de Pelles
Forgeur d'Epées
Forgeur de Dagues
Forgeur de Haches
Forgeur de Marteaux
Forgeur de Pelles
Joaillomage
Mineur
Paysan


In [71]:
recipes_df = pd.DataFrame(all_recipes).set_index("object_id")
recipes_df.to_json("recipes.json")
recipes_df

,detail_url,object_name,link,level,profession,ingredients
object_id,,,,,,
1642,/fr/mmorpg/encyclopedie/objets/recette/1642,Potion d'oubli de métier : Forgemage de Dagues,/fr/mmorpg/encyclopedie/consommables/1642-poti...,1,Alchimiste,"[{'count': '1', 'link': '/fr/mmorpg/encycloped..."
1643,/fr/mmorpg/encyclopedie/objets/recette/1643,Potion d'oubli de métier : Forgemage de Haches,/fr/mmorpg/encyclopedie/consommables/1643-poti...,1,Alchimiste,"[{'count': '2', 'link': '/fr/mmorpg/encycloped..."
1644,/fr/mmorpg/encyclopedie/objets/recette/1644,Potion d'oubli de métier : Forgemage de Marteaux,/fr/mmorpg/encyclopedie/consommables/1644-poti...,1,Alchimiste,"[{'count': '2', 'link': '/fr/mmorpg/encycloped..."
1645,/fr/mmorpg/encyclopedie/objets/recette/1645,Potion d'oubli de métier : Forgemage d'Epées,/fr/mmorpg/encyclopedie/consommables/1645-poti...,1,Alchimiste,"[{'count': '1', 'link': '/fr/mmorpg/encycloped..."
1646,/fr/mmorpg/encyclopedie/objets/recette/1646,Potion d'oubli de métier : Forgemage de Pelles,/fr/mmorpg/encyclopedie/consommables/1646-poti...,1,Alchimiste,"[{'count': '1', 'link': '/fr/mmorpg/encycloped..."
...,...,...,...,...,...,...
6672,/fr/mmorpg/encyclopedie/objets/recette/6672,Farine de Xavier le Boulanger,/fr/mmorpg/encyclopedie/ressources/6672-farine...,100,Paysan,"[{'count': '2', 'link': '/fr/mmorpg/encycloped..."
11194,/fr/mmorpg/encyclopedie/objets/recette/11194,Farine de Frostiz,/fr/mmorpg/encyclopedie/ressources/11194-farin...,100,Paysan,"[{'count': '2', 'link': '/fr/mmorpg/encycloped..."
11199,/fr/mmorpg/encyclopedie/objets/recette/11199,Graine de Frostiz,/fr/mmorpg/encyclopedie/ressources/11199-grain...,100,Paysan,"[{'count': '5', 'link': '/fr/mmorpg/encycloped..."


In [72]:
professions_df = pd.DataFrame(professions).T
professions_df.to_json("professions.json")
professions_df

,link,recipes
Alchimiste,/fr/mmorpg/encyclopedie/metiers/26-alchimiste,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Bijoutier,/fr/mmorpg/encyclopedie/metiers/16-bijoutier,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Boucher,/fr/mmorpg/encyclopedie/metiers/56-boucher,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Boulanger,/fr/mmorpg/encyclopedie/metiers/25-boulanger,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Bricoleur,/fr/mmorpg/encyclopedie/metiers/65-bricoleur,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Bûcheron,/fr/mmorpg/encyclopedie/metiers/2-bucheron,[]
Chasseur,/fr/mmorpg/encyclopedie/metiers/41-chasseur,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Cordomage,/fr/mmorpg/encyclopedie/metiers/62-cordomage,[]
Cordonnier,/fr/mmorpg/encyclopedie/metiers/15-cordonnier,[{'detail_url': '/fr/mmorpg/encyclopedie/objet...
Costumage,/fr/mmorpg/encyclopedie/metiers/64-costumage,[]


In [73]:
{"Age":12,
"Cha":9,
"de chasse":394,
"Fo":12,
"Ine":22,
"Ini":159,
"Prospe":1977,
"Ré Air":193,
"Ré Eau":19,
"Ré Feu":19,
"Ré Neutre":79,
"Ré Per air":166,
"Ré Per Eau":157,
"Ré Per Feu":932,
"Ré Per Neutre":498,
"Ré Per Terre":898,
"Ré Terre":390,
"Sa":16,
"Vi":45,
"Cri":99,
"Do":452,
"Do Air":374,
"Do Cri":792,
"Do Eau":189,
"Do Feu":203,
"Do Neutre":284,
"Do Pou":252,
"Do Terre":157,
"Rui":49,
"Invo":7556,
"Pa Age":88,
"Pa Cha":76,
"Pa Do Air":7575,
"Pa Do Cri":3631,
"Pa Do Eau":5749,
"Pa Do Feu":4967,
"Pa Do Neutre":896,
"Pa Do Pou":3902,
"Pa Do Terre":2723,
"Pa Fo":108,
"Pa Fui":184,
"Pa Ine":121,
"Pa Ini":2160,
"Pa Prospe":8889,
"Pa Ré Cri":15956,
"Pa Ré Pa":26939,
"Pa Ré Pme":12077,
"Pa Ré Pou":4563,
"Pa Ret Pa":2267,
"Pa Ret Pme":833,
"Pa Sa":180,
"Pa Tac":1116,
"Pa Vi":777,
"Po":9911,
"Pod":28,
"Pui":96,
"Ré Cri":1404,
"Ré Pa":14894,
"Ré Pme":2304,
"Ré Pou":1962,
"Ret Pa":842,
"Ret Pme":202,
"So":188,
"Tac":144,
"Pa Pod":224,
"Pa Pui":1007,
"Ra Age":669,
"Ra Cha":625,
"Ra Fo":958,
"Ra Ine":1104,
"Ra Ini":8810,
"Ra Pod":1190,
"Ra Pui":7415,
"Ra Sa":823,
"Ra Vi":8552,
"Pa Ré Air":240,
"Pa Ré Eau":185,
"Pa Ré Feu":142,
"Pa Ré Neutre":1742,
"Pa Ré Terre":3200,
"Ga Pa":106910,
"Ga Pme":104554,
"Pa So":400}

{'Age': 12,
 'Cha': 9,
 'de chasse': 394,
 'Fo': 12,
 'Ine': 22,
 'Ini': 159,
 'Prospe': 1977,
 'Ré Air': 193,
 'Ré Eau': 19,
 'Ré Feu': 19,
 'Ré Neutre': 79,
 'Ré Per air': 166,
 'Ré Per Eau': 157,
 'Ré Per Feu': 932,
 'Ré Per Neutre': 498,
 'Ré Per Terre': 898,
 'Ré Terre': 390,
 'Sa': 16,
 'Vi': 45,
 'Cri': 99,
 'Do': 452,
 'Do Air': 374,
 'Do Cri': 792,
 'Do Eau': 189,
 'Do Feu': 203,
 'Do Neutre': 284,
 'Do Pou': 252,
 'Do Terre': 157,
 'Rui': 49,
 'Invo': 7556,
 'Pa Age': 88,
 'Pa Cha': 76,
 'Pa Do Air': 7575,
 'Pa Do Cri': 3631,
 'Pa Do Eau': 5749,
 'Pa Do Feu': 4967,
 'Pa Do Neutre': 896,
 'Pa Do Pou': 3902,
 'Pa Do Terre': 2723,
 'Pa Fo': 108,
 'Pa Fui': 184,
 'Pa Ine': 121,
 'Pa Ini': 2160,
 'Pa Prospe': 8889,
 'Pa Ré Cri': 15956,
 'Pa Ré Pa': 26939,
 'Pa Ré Pme': 12077,
 'Pa Ré Pou': 4563,
 'Pa Ret Pa': 2267,
 'Pa Ret Pme': 833,
 'Pa Sa': 180,
 'Pa Tac': 1116,
 'Pa Vi': 777,
 'Po': 9911,
 'Pod': 28,
 'Pui': 96,
 'Ré Cri': 1404,
 'Ré Pa': 14894,
 'Ré Pme': 2304,
 'Ré Pou': 19

In [76]:
items

[<div class="ak-list-element">
 <div class="ak-front">
     1 x  </div>
 <div class="ak-main">
 <div class="ak-main-content">
 <div class="ak-image">
 <a href="/fr/mmorpg/encyclopedie/ressources/11499-frostiz-souffle">
 <span class="ak-linker"><img height="48" src="https://static.ankama.com/dofus-touch/www/img/../../../dofus-touch/www/game/items/52/34553.w48h48.png" width="48"/></span><script type="application/json">{"iShowDelay":500,"iHideDelay":250,"linker-id":"linker_item_11499","linker-path":"\/fr\/linker\/item","linker-display-type":"TOOLTIP","linker-query-datas":{"l":"fr","id":"11499"}}</script> </a>
 </div>
 <div class="ak-content">
 <div class="ak-title">
 <a href="/fr/mmorpg/encyclopedie/ressources/11499-frostiz-souffle">
 <span class="ak-linker">Frostiz soufflé</span><script type="application/json">{"iShowDelay":500,"iHideDelay":250,"linker-id":"linker_item_11499","linker-path":"\/fr\/linker\/item","linker-display-type":"TOOLTIP","linker-query-datas":{"l":"fr","id":"11499"}}<

In [83]:
effects = items["effects"].explode()
effects = effects.dropna().drop_duplicates()
effect_names = pd.DataFrame(list(effects))["name"]
list(effect_names)

['vi',
 'fo',
 'ag',
 'ch',
 'sa',
 'daf',
 'dtf',
 'dnf',
 'def',
 'rtp',
 'cc',
 'po',
 'pa',
 'ii',
 'vi',
 'fo',
 'rc',
 'sa',
 'ta',
 'dtf',
 'rep',
 'rfp',
 'cc',
 'pa',
 'vi',
 'ag',
 'sa',
 'ta',
 'daf',
 'rtp',
 'rep',
 'dc',
 'cc',
 'po',
 'pa',
 'pm',
 'vi',
 'in',
 'rp',
 'sa',
 'fu',
 'dff',
 'rnp',
 'rep',
 'cc',
 'po',
 'pa',
 'vi',
 'ag',
 'sa',
 'pp',
 'fu',
 'daf',
 'dp',
 'rtp',
 'cc',
 'pa',
 'vi',
 'pu',
 'rp',
 'rc',
 'cc',
 'pa',
 'sa',
 'vi',
 'in',
 'ch',
 'rp',
 'sa',
 'fu',
 'dff',
 'def',
 'rtp',
 'rap',
 'rpm',
 'ic',
 'pm',
 'vi',
 'pu',
 'rep',
 'rfp',
 'pm',
 'sa',
 'vi',
 'pu',
 'rc',
 'sa',
 'pp',
 'rtp',
 'rep',
 'epa',
 'cc',
 'pm',
 'pa',
 'vi',
 'fo',
 'ag',
 'sa',
 'ta',
 'daf',
 'dtf',
 'rfp',
 'po',
 'pm',
 'pa',
 'vi',
 'fo',
 'rc',
 'sa',
 'dtf',
 'ta',
 'rep',
 'rfp',
 'epm',
 'ic',
 'pm',
 'sp',
 'vi',
 'ta',
 'epm',
 'rt',
 're',
 'vi',
 'in',
 'ag',
 'ch',
 'sa',
 'dff',
 'daf',
 'def',
 'po',
 'so',
 'fu',
 'rpa',
 'rnp',
 'rtp',
 'rn',
 

In [86]:
ingredients = items["ingredients"].explode()
ingredients = ingredients.dropna()
ingredients = pd.DataFrame(list(ingredients)).set_index("id")
ingredients = ingredients.drop(columns=["count", "picture", "item_id"])
ingredients = ingredients.drop_duplicates()
ingredients.to_json("ingredients.json")
ingredients

,name
id,
11883,Condyle de Fuji Givrefoux
11939,Incisive de Glourmand
13062,Galet acajou
13994,Scapula du Comte Harebourg
15748,Orbe irisé
...,...
285,Farine de Blé
694,Dofus Pourpre
737,Dofus Émeraude


: 